In [8]:
from torch_geometric.loader import DataLoader, ImbalancedSampler
from torch_geometric.transforms import FaceToEdge, OneHotDegree
import torchvision.transforms as transforms

from mantra.simplicial import SimplicialDataset
from mantra.transforms import (
    TriangulationToFaceTransform,
    OrientableToClassTransform,
    DegreeTransform,
)
from validation.validate_homology import validate_betti_numbers

import torch

In [9]:
tr = transforms.Compose(
    [
        TriangulationToFaceTransform(),
        FaceToEdge(remove_faces=False),
        DegreeTransform(),
        OrientableToClassTransform(),
        OneHotDegree(max_degree=12,cat=False)
    ]
)

dataset = SimplicialDataset(root="./data", transform=tr)


print()
print(f"Dataset: {dataset}:")
print("====================")
print(f"Number of graphs: {len(dataset)}")
print(f"Number of features: {dataset.num_features}")
print(f"Number of classes: {dataset.num_classes}")

data = dataset[0]  # Get the first graph object.

print()
print(data)
print("=============================================================")

# Gather some statistics about the first graph.
print(f"Number of nodes: {len(data.x)}")
print(f"Number of edges: {data.num_edges}")
print(f"Average node degree: {data.num_edges / len(data.x):.2f}")
print(f"Has isolated nodes: {data.has_isolated_nodes()}")
print(f"Has self-loops: {data.has_self_loops()}")
print(f"Is undirected: {data.is_undirected()}")

print("=============================================================")
print(f"Number of orientable Manifolds: {sum(dataset.orientable)}")
print(
    f"Number of non-orientable Manifolds: {len(dataset) - sum(dataset.orientable)}"
)
print(
    f"Percentage: {sum(dataset.orientable) / len(dataset):.2f}, {(len(dataset) - sum(dataset.orientable)) / len(dataset):.2f}"
)


Dataset: SimplicialDataset(712):
Number of graphs: 712
Number of features: 13
Number of classes: 2

Data(dimension=[1], n_vertices=[1], torsion_coefficients=[3], betti_numbers=[3], orientable=[1], genus=[1], name='S^2', face=[3, 4], edge_index=[2, 12], x=[4, 13], y=[1])
Number of nodes: 4
Number of edges: 12
Average node degree: 3.00
Has isolated nodes: False
Has self-loops: False
Is undirected: True
Number of orientable Manifolds: 193
Number of non-orientable Manifolds: 519
Percentage: 0.27, 0.73


In [10]:
torch.tensor([data.y for data in dataset])

tensor([1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1,
        1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
        1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [11]:
dataset = dataset.shuffle()

train_dataset = dataset[:-150]
test_dataset = dataset[-150:]

print(f"Number of training graphs: {len(train_dataset)}")
print(f"Number of test graphs: {len(test_dataset)}")


Number of training graphs: 562
Number of test graphs: 150


In [12]:
train_loader = DataLoader(train_dataset,batch_size=10)#,sampler=ImbalancedSampler(train_dataset))
test_loader = DataLoader(test_dataset,batch_size=10)


for batch in train_loader:
    break

batch.y

tensor([0, 0, 0, 0, 0, 0, 1, 0, 1, 0])

In [13]:
from torch.nn import Linear
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
from torch_scatter import segment_coo

class PermInvariant(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        # torch.manual_seed(12345)
        self.classification = nn.Sequential( 
            nn.Linear(dataset.num_node_features,hidden_channels),
            nn.ReLU(),
            nn.Linear(hidden_channels,hidden_channels),
            nn.ReLU(),
            nn.Linear(hidden_channels,2),
            nn.ReLU()
        )

    def forward(self, batch):
        x = self.classification(batch.x)
        # print(batch.x)
        # print(x)
        return segment_coo(x,batch.batch,reduce="sum")


model = PermInvariant(hidden_channels=64)


In [14]:
model = PermInvariant(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.CrossEntropyLoss()


def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
        out = model(data
        )  # Perform a single forward pass.
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.


def test(loader):
    model.eval()

    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data)
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int(
            (pred == data.y).sum()
        )  # Check against ground-truth labels.
    return correct / len(
        loader.dataset
    )  # Derive ratio of correct predictions.


for epoch in range(1, 40):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(
        f"Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}"
    )

Epoch: 001, Train Acc: 0.7278, Test Acc: 0.7333
Epoch: 002, Train Acc: 0.7278, Test Acc: 0.7333
Epoch: 003, Train Acc: 0.7278, Test Acc: 0.7333
Epoch: 004, Train Acc: 0.7278, Test Acc: 0.7333
Epoch: 005, Train Acc: 0.7278, Test Acc: 0.7333
Epoch: 006, Train Acc: 0.7278, Test Acc: 0.7333
Epoch: 007, Train Acc: 0.7278, Test Acc: 0.7333
Epoch: 008, Train Acc: 0.7278, Test Acc: 0.7333
Epoch: 009, Train Acc: 0.7278, Test Acc: 0.7333
Epoch: 010, Train Acc: 0.7278, Test Acc: 0.7333
Epoch: 011, Train Acc: 0.7278, Test Acc: 0.7333
Epoch: 012, Train Acc: 0.7278, Test Acc: 0.7333
Epoch: 013, Train Acc: 0.7278, Test Acc: 0.7333
Epoch: 014, Train Acc: 0.7278, Test Acc: 0.7333
Epoch: 015, Train Acc: 0.7278, Test Acc: 0.7333


KeyboardInterrupt: 

In [ ]:
model(batch)[:,0]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], grad_fn=<SelectBackward0>)